In [56]:
import sys
sys.path.insert(0, '../tools')

from pprint import pprint
%load_ext prov_magics
import prov_dictionary
from prov_parser import build_parser
from qmutable_prov import *

The prov_magics extension is already loaded. To reload it, use:
  %reload_ext prov_magics


In [57]:
from dot import _unquote

def extract_entity(eid, attrs={}, id_=None):
    return ["entity", [eid], attrs, id_]

def extract_value(vid, attrs={}, id_=None):
    return ["value", [vid], attrs, id_]


parser = build_parser({
    'prefix': lambda *args, **kwargs: None,
    'entity': extract_entity,
    'value': extract_value,
})



def ext(text, arg):
    node = parser(text)[0]
    if arg.startswith("_"):
        return node[1][int(arg[1:])]
    if arg == "id_":
        return node[-1]
    if arg in node[2]:
        return node[2][arg]
    return None

def attr(text, arg):
    return _unquote(ext(text, arg))
            
def update(text, **kwargs):
    node = parser(text)[0]
    for arg, value in kwargs.items():
        if arg.startswith("_"):
            node[1][int(arg[1:])] = value
        elif arg == "id_":
            node[-1] = value
        elif arg.startswith("d__"):
            del node[2][arg]
        else:
            node[2][arg] = value
    return querier.text(*node)



In [62]:
querier.load("../mutable_prov/temp/L2__list_definition0.provn");
querier.generate('''
value(v10000, [repr="10000"])
entity(e10000, [value="10000", type="literal"])
defined(e10000, v10000, 2018-02-26T14:41:52.080421)
''')

[['v10000', 'value(v10000, [repr="10000"])'],
 ['e10000', 'entity(e10000, [value="10000", type="literal"])'],
 ['e10000',
  'v10000',
  datetime.datetime(2018, 2, 26, 14, 41, 52, 80421),
  'defined(e10000, v10000, 2018-02-26T14:41:52.080421)']]

In [63]:
querier.load("../mutable_prov/floydwarshall.provn");

In [94]:
from collections import defaultdict
from copy import copy
all_states = defaultdict(list)
derivations = defaultdict(list)

E, V, C, T, TX = var(5)
for _ in derived_by_insertion(E, V, C, T, TX):
    derivations[V.bound].append([T.bound, E.bound, C.bound, TX.bound])

    
for val, der in derivations.items():
    der.sort()
    obj = {}
    for time, entity, changes, text in der:
        obj = copy(obj)
        for key, value in changes:
            obj[_unquote(key)] = value
        all_states[val].append((time, obj))
    
all_states['row1']



[(datetime.datetime(2018, 2, 26, 14, 41, 52, 81023),
  {'0': 'v10000', '1': 'v0', '2': 'v2'}),
 (datetime.datetime(2018, 2, 26, 14, 42, 2, 150059),
  {'0': 'vsum_21', '1': 'v0', '2': 'v2'})]

In [95]:
all_states['list_1']

[(datetime.datetime(2018, 2, 26, 14, 41, 52, 81023),
  {'0': 'row0', '1': 'row1', '2': 'row2'})]

In [96]:
new_all_states = defaultdict(list)
for val, states in all_states.items():
    new_states = []
    for time, obj in states:
        new_states.append((time, obj))
        changes = []
        for key, sub in obj.items():
            if sub in all_states:
                for x in all_states[sub]:
                    if x[0] > time:
                        changes.append([x[0], key, x[1]])
        changes.sort()
        for time, k, v in changes:
            new_states.append((time, obj))
    new_states.sort()
    new_all_states[val] = new_states

In [97]:
new_all_states

defaultdict(list,
            {'list_1': [(datetime.datetime(2018, 2, 26, 14, 41, 52, 81023),
               {'0': 'row0', '1': 'row1', '2': 'row2'}),
              (datetime.datetime(2018, 2, 26, 14, 41, 55, 929679),
               {'0': 'row0', '1': 'row1', '2': 'row2'}),
              (datetime.datetime(2018, 2, 26, 14, 41, 57, 845557),
               {'0': 'row0', '1': 'row1', '2': 'row2'}),
              (datetime.datetime(2018, 2, 26, 14, 42, 2, 150059),
               {'0': 'row0', '1': 'row1', '2': 'row2'})],
             'row0': [(datetime.datetime(2018, 2, 26, 14, 41, 52, 81023),
               {'0': 'v0', '1': 'v1', '2': 'v4'}),
              (datetime.datetime(2018, 2, 26, 14, 41, 57, 845557),
               {'0': 'v0', '1': 'v1', '2': 'vsum_11'})],
             'row1': [(datetime.datetime(2018, 2, 26, 14, 41, 52, 81023),
               {'0': 'v10000', '1': 'v0', '2': 'v2'}),
              (datetime.datetime(2018, 2, 26, 14, 42, 2, 150059),
               {'0': 'vsum_21', '

In [55]:
import importlib
import annotations as prov
importlib.reload(prov)

def add_unmarked(marked, fn):
    TX = var()
    for _ in fn(text=TX):
        text = TX.bound
        if text not in marked:
            prov.add(text)

prov.reset_prov("../mutable_prov/unfold/", False)


marked = set()
E, V, C, T, TX = var(5)
valuemap = {
    V.bound: TX.bound
    for _ in value(V, TX)
}

value_with_parts = {
    V.bound: (E.bound, V.bound, C.bound, T.bound, TX.bound)
    for _ in derived_by_insertion(E, V, C, T, TX)
}

value_without_parts = set(valuemap) - set(value_with_parts) 

value_to_entity = {}

# Add imutable values
for vid in value_without_parts:
    vtext = valuemap[vid]
    for _ in entity(E, TX) & accessed(E, vid):
        marked.add(TX.bound)
        prov.entity(ext(TX.bound, "_0"), attr(vtext, "repr"), attr(TX.bound, "type"))
        value_to_entity[vid] = E.bound
    for _ in entity(E, TX) & defined(E, vid):
        marked.add(TX.bound)
        prov.entity(ext(TX.bound, "_0"), attr(vtext, "repr"), attr(TX.bound, "type"))
        value_to_entity[vid] = E.bound
    
# List definition
for _ in defined(E, V, T) & derived_by_insertion(E, V, C, T) & entity(E, TX):
    vtext = valuemap[V.bound]
    list_id = ext(TX.bound, "_0")
    list_value = attr(vtext, "repr")
    der = []
    parts = {}
    whole = prov.entity(list_id, list_value, "Dictionary")
    empty = prov.entity("e" + list_id, "{}", "EmptyDictionary")
    for key, vid in C.bound:
        key = _unquote(key)
        member = prov.entity(
            list_id[:-1] + key + "_" + list_id[-1], 
            attr(valuemap[vid], "repr"), "number")
        der.append((member, value_to_entity[vid]))
        parts[key] = member
    
    prov.derivedByInsertionFrom(whole, empty, parts)
    prov.activity("definelist", der, generated=[whole])
    marked.add(TX.bound)
    
            

# Add unmarked PROV
add_unmarked(marked, entity)
add_unmarked(marked, activity)
add_unmarked(marked, used)
add_unmarked(marked, was_derived_from)
add_unmarked(marked, was_generated_by)
add_unmarked(marked, had_member)
#print("\n".join(prov.RESULT))
pprint(prov.stats("../mutable_prov/unfold/result", "provn svg"))

KeyError: 'row0'

In [ ]:
marked